In [2]:
%matplotlib notebook

In [3]:
import numpy as np
import cv2 as cv
import glob
 
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
objp=0.008*objp

In [ ]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('*.jpg')
 
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (5,5), (-1,-1), criteria)
        imgpoints.append(corners2)
    # Draw and display the corners
    cv.drawChessboardCorners(img, (7,7), corners2, ret)
    cv.imshow('img', img)
    cv.waitKey(500)

cv.destroyAllWindows()

print("hello，World\n")
print(images)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print(mtx);

In [ ]:
print(images);

In [1]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

NameError: name 'cv' is not defined

In [ ]:
mtx

In [ ]:
dist

In [1]:
#去畸变

In [2]:
img = cv.imread(images[0])
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

NameError: name 'cv' is not defined

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

In [3]:
images[0]

NameError: name 'images' is not defined

In [4]:
#rvecs, tvecs的作用

In [5]:
def draw(img, corners, imgpts):
 corner = tuple(corners[0].ravel().astype(int))
 img = cv.line(img, corner , tuple(imgpts[0].ravel().astype(int)), (255,0,0), 5)
 img = cv.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0,255,0), 5)
 img = cv.line(img, corner , tuple(imgpts[2].ravel().astype(int)), (0,0,255), 5)
 return img

In [6]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
 
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

NameError: name 'cv' is not defined

In [7]:
img = cv.imread(images[0])
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, corners = cv.findChessboardCorners(gray, (7,7),None)
 
if ret == True:
    corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

# Find the rotation and translation vectors.
ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)

# project 3D points to image plane
imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)

img = draw(img,corners2 ,imgpts)
cv.imshow('img',img)
k = cv.waitKey(0) & 0xFF
if k == ord('s'):
    cv.imwrite(fname[:6]+'.png', img)
 
cv.destroyAllWindows()

NameError: name 'cv' is not defined

In [8]:
rvecs, tvecs

NameError: name 'rvecs' is not defined

In [2]:
print("hello,world\n")

hello,world

